In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model,load_model,Sequential
from tensorflow.keras.preprocessing.image import load_img,save_img,img_to_array
from tensorflow.keras.layers import Dense, Dropout, Flatten,Conv2D,MaxPooling2D,BatchNormalization
from keras.callbacks import ModelCheckpoint
from tensorflow import keras
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras.models import load_model

In [ ]:
batch = 128
num_classes = 200
epochs = 40
input_shape = (224,224,3)

In [ ]:
data = ImageDataGenerator(rescale = 1./255,rotation_range=20,horizontal_flip=True,width_shift_range = 0.3,height_shift_range=0.3)
traindata = data.flow_from_directory('/kaggle/input/image-detect/train',target_size=(224,224),batch_size=batch,class_mode = "categorical",shuffle=True) # replace with your path

In [ ]:
val_txt = pd.read_csv(r'/kaggle/input/image-detect/val/val_annotations.txt',sep='\t',header= None) # replace with your path
val_txt = val_txt.drop(val_txt.columns[[2,3,4,5]],axis =1)
val_df = pd.DataFrame(data=val_txt)
data_ = ImageDataGenerator(rescale = 1./255)
val_data = data_.flow_from_dataframe(val_df,directory = '/kaggle/input/image-detect/val/images',x_col = val_df.columns[0],y_col=val_df.columns[1],target_size = (224,224),color_mode="rgb",class_mode="categorical")

In [ ]:
labels= traindata.class_indices

In [ ]:
pip install image-classifiers==0.2.2

In [ ]:
pip install image-classifiers==1.0.0b1

In [ ]:
pip install git+https://github.com/qubvel/classification_models.git

In [ ]:
from classification_models.keras import Classifiers
ResNet18, preprocess_input = Classifiers.get('resnet18')

In [ ]:
base_model = ResNet18((224, 224, 3), weights='imagenet',include_top = False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(200, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(traindata,validation_data = val_data,epochs = 20)

In [ ]:
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('ResNet 18 accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('ResNet 18 loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.show()